In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.7.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 5s 1us/step


In [5]:
train_images = train_images/255.0
test_images = test_images/255.0

In [6]:
len(train_images)

60000

In [7]:
len(test_images)

10000

In [8]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [10]:
train_images.shape

(60000, 28, 28, 1)

**Keras Tuner Implementation**

In [11]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [12]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

C:\Users\USER\AppData\Local\Temp/ipykernel_2236/556418634.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [13]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=3,directory='output',project_name="Mnist Fashion")

In [14]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 16m 09s]
val_accuracy: 0.906166672706604

Best val_accuracy So Far: 0.9110000133514404
Total elapsed time: 01h 04m 55s
INFO:tensorflow:Oracle triggered exit


In [15]:
model = tuner_search.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 112)       2912      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        32288     
                                                                 
 flatten (Flatten)           (None, 15488)             0         
                                                                 
 dense (Dense)               (None, 112)               1734768   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 1,771,098
Trainable params: 1,771,098
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(train_images, train_labels, epochs = 10, validation_split = 0.3)

Epoch 1/10
1313/1313 [==============================] - 155s 116ms/step - loss: 0.1699 - accuracy: 0.9374 - val_loss: 0.2032 - val_accuracy: 0.9285
Epoch 2/10
1313/1313 [==============================] - 149s 114ms/step - loss: 0.1344 - accuracy: 0.9493 - val_loss: 0.2160 - val_accuracy: 0.9277
Epoch 3/10
1313/1313 [==============================] - 154s 117ms/step - loss: 0.1000 - accuracy: 0.9643 - val_loss: 0.2450 - val_accuracy: 0.9184
Epoch 4/10
1313/1313 [==============================] - 168s 128ms/step - loss: 0.0758 - accuracy: 0.9723 - val_loss: 0.2472 - val_accuracy: 0.9222
Epoch 5/10
1313/1313 [==============================] - 163s 124ms/step - loss: 0.0597 - accuracy: 0.9774 - val_loss: 0.3186 - val_accuracy: 0.9220
Epoch 6/10
1313/1313 [==============================] - 158s 121ms/step - loss: 0.0442 - accuracy: 0.9840 - val_loss: 0.3385 - val_accuracy: 0.9249
Epoch 7/10
1313/1313 [==============================] - 158s 121ms/step - loss: 0.0378 - accuracy: 0.9861 - val_